In [1]:
import logging

import probtorch
import torch

import combinators
import importance
import mcmc
import smc
import ssm
import utils

In [2]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [3]:
generative = combinators.ParticleTrace(1)

In [4]:
mu = torch.zeros(1)
sigma = torch.ones(1) / 2
delta = torch.ones(1)
zs = torch.ones(50+1) * -1
zs[0] = 0

In [5]:
ssm_step = combinators.Model(ssm.ssm_step)

In [6]:
for t in range(zs.shape[0] - 1):
    zs[t+1], mu, sigma, delta = ssm_step((zs[t], mu, sigma, delta), t, trace=generative)

In [7]:
data = generative.unwrap(lambda k, rv: 'X_' in k)

In [8]:
num_particles = 100

In [9]:
ssm_params = {
    'mu': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    },
    'sigma': {
        'loc': torch.tensor(1.),
        'scale': torch.tensor(0.25),
    },
    'delta': {
        'loc': torch.tensor(0.),
        'scale': torch.tensor(0.25),
    }
}

In [10]:
init_ssm_generative = combinators.Model(ssm.init_ssm, hyper=ssm_params)
smc_ssm = smc.SequentialMonteCarlo(combinators.Model(ssm.ssm_step), 50, initializer=init_ssm_generative)

In [11]:
init_ssm_proposal = combinators.Model(ssm.init_ssm, trainable=ssm_params)
ssm_step_proposal = combinators.Model(ssm.ssm_step)
ssm_proposal = combinators.Model.compose(combinators.Model.sequence(ssm_step_proposal, 50),
                                         init_ssm_proposal, intermediate_name='initializer')

In [12]:
ssm_importance = importance.ImportanceSampler(model=smc_ssm, proposal=ssm_proposal)

In [ ]:
inference, init_ssm_params = smc.variational_smc(num_particles, ssm_importance, 500, data,
                                                 use_cuda=True, lr=1e-1)

> /home/eli/AnacondaProjects/combinators/smc.py(73)variational_smc()
-> sampler.simulate(trace=ResamplerTrace(num_particles, data=data),
OrderedDict([('X_1', tensor(3.3037)), ('X_2', tensor(1.0082)), ('X_3', tensor(1.2351)), ('X_4', tensor(2.9516)), ('X_5', tensor(2.7404)), ('X_6', tensor(5.9015)), ('X_7', tensor(5.8797)), ('X_8', tensor(5.9043)), ('X_9', tensor(6.5513)), ('X_10', tensor(10.1426)), ('X_11', tensor(9.4824)), ('X_12', tensor(8.0535)), ('X_13', tensor(7.1640)), ('X_14', tensor(9.1269)), ('X_15', tensor(10.1310)), ('X_16', tensor(7.9248)), ('X_17', tensor(10.8658)), ('X_18', tensor(13.4717)), ('X_19', tensor(11.5273)), ('X_20', tensor(12.4471)), ('X_21', tensor(13.6567)), ('X_22', tensor(13.9297)), ('X_23', tensor(15.6730)), ('X_24', tensor(12.0968)), ('X_25', tensor(13.8379)), ('X_26', tensor(14.0255)), ('X_27', tensor(17.8572)), ('X_28', tensor(20.2927)), ('X_29', tensor(21.4752)), ('X_30', tensor(24.0168)), ('X_31', tensor(24.1006)), ('X_32', tensor(24.0562)), ('X_33', 

In [ ]:
for t in range(1, zs.shape[0]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

In [ ]:
particle_mh = mcmc.IndependentMH(smc_ssm, ssm_proposal, 1000)

In [ ]:
samples, elbos = particle_mh(trace=importance.ResamplerTrace(num_particles, data=data))
inference = particle_mh.trace

In [ ]:
inference

In [ ]:
generative

In [ ]:
for t in range(1, zs.shape[0]):
    key = 'Z_%d' % t
    accuracy = ((inference[key].value - utils.optional_to(generative[key], inference[key].value))**2).mean()
    print('SMC MSE at time %d: %f' % (t, accuracy))

In [ ]:
elbos